In [1]:
import pandas as pd

In [2]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Nevada - Metadata/meta-Nevada.json"
business_df = pd.read_json(path, lines = True)

In [3]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Nevada - Review/review-Nevada.json"
review_df = pd.read_json(path, lines = True)

In [4]:
business_agg = business_df.groupby("gmap_id", as_index=False).agg({
    "name": "first",  # Keep first occurrence (assumes same name per gmap_id)
    "address": "first",
    "description": "first",
    "latitude": "first",
    "longitude": "first",
    "category": lambda x: list(set(x.sum())) if x.apply(lambda v: isinstance(v, list)).any() else list(set(x)),
    "avg_rating": "mean",  # Use mean rating if multiple exist
    "num_of_reviews": "sum",
    "price": "first",
    "hours": "first",
    "MISC": "first",
    "state": "first",
    "relative_results": "first",
    "url": "first"
})

In [6]:
merged_df = review_df.merge(business_agg, on="gmap_id", how="left")

In [7]:
#remove rows/records where 'text' is null
merged_df = merged_df.dropna(subset = ['text'])
#check if null values in 'text' column still exist
print(merged_df['text'].isna().sum())

0


In [8]:
merged_df = merged_df.drop(columns = ['pics'])

In [9]:
negative_df = merged_df[merged_df['rating'].isin([1,2])]
negative_df

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
9,1.149544e+20,So_ Blind,1573159413820,1.0,Save yourself the frustration and just ignore ...,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,-115.176338,[Coffee shop],3.2,48,None,"[[Thursday, 6AM–2PM], [Friday, 6AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x80c8c5c561857ea1:0x1649d82cf55356b1, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
11,1.086896e+20,Melissa Demidov,1561477792923,1.0,"8:05 am I get there, sign says CLOSED even th...",None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,-115.176338,[Coffee shop],3.2,48,None,"[[Thursday, 6AM–2PM], [Friday, 6AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x80c8c5c561857ea1:0x1649d82cf55356b1, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
12,1.129924e+20,Lisa Koehler,1571294526703,1.0,"I just wanted some iced tea and the lady said,...",None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,-115.176338,[Coffee shop],3.2,48,None,"[[Thursday, 6AM–2PM], [Friday, 6AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x80c8c5c561857ea1:0x1649d82cf55356b1, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
13,1.007051e+20,Rudy Diaz,1581471097395,1.0,Stopped on the way out. The coffee was weak. ...,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,-115.176338,[Coffee shop],3.2,48,None,"[[Thursday, 6AM–2PM], [Friday, 6AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x80c8c5c561857ea1:0x1649d82cf55356b1, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
16,1.066181e+20,Thomas Charley,1553614482083,1.0,Person behind counter was rude ....,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,-115.176338,[Coffee shop],3.2,48,None,"[[Thursday, 6AM–2PM], [Friday, 6AM–2PM], [Satu...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 2PM,"[0x80c8c5c561857ea1:0x1649d82cf55356b1, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833206,1.111813e+20,M. Del,1511403706460,2.0,Nice room but terrible night sleep. Continuou...,None,0x80c8c43b8995c0b5:0xe2ac56cdb7db8765,SB Las Vegas at Caesars Palace,"SB Las Vegas at Caesars Palace, Paradise, NV 8...",None,...,-115.173078,[Bus stop],4.5,78,None,None,"{'Accessibility': ['Passenger loading area'], ...",None,"[0x80c8c43c44cae095:0x77c9c8f2a2af3ff7, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
8833207,1.048418e+20,喵咪,1514964796392,1.0,Poor guest service. Waited for our luggages fo...,None,0x80c8c43b8995c0b5:0xe2ac56cdb7db8765,SB Las Vegas at Caesars Palace,"SB Las Vegas at Caesars Palace, Paradise, NV 8...",None,...,-115.173078,[Bus stop],4.5,78,None,None,"{'Accessibility': ['Passenger loading area'], ...",None,"[0x80c8c43c44cae095:0x77c9c8f2a2af3ff7, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
8833285,1.026047e+20,Kevin Diaz,1592202676820,1.0,Bad bottle of wine for $25,None,0x80c8c47be23c6541:0xe6c010eb30ac72b1,Mom and Pop's Liquor Store,"Mom and Pop's Liquor Store, 557 E Sahara Ave #...",None,...,-115.148588,"[Liquor store, Beer store]",4.4,145,None,"[[Wednesday, 8AM–10PM], [Thursday, 8AM–10PM], ...","{'Service options': ['In-store shopping', 'Del...",None,"[0x80c8c3785c181bdb:0xd9672050801b24e5, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...
8833291,1.139538e+20,Teresa John,1566173405294,

In [10]:
import ast

negative_df["category"] = negative_df["category"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expand the list into separate columns
category_expanded = negative_df["category"].apply(pd.Series)

# Rename columns as 'category_1', 'category_2', etc.
category_expanded.columns = [f"category_{i+1}" for i in range(category_expanded.shape[1])]

# Concatenate with original dataset
negative_df = pd.concat([negative_df, category_expanded], axis=1)

# Drop original 'category' column
#cleaned_file.drop(columns=["category"], inplace=True)

negative_df

C:\Users\eliza\AppData\Local\Temp\ipykernel_21560\761791609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df["category"] = negative_df["category"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11
9,1.149544e+20,So_ Blind,1573159413820,1.0,Save yourself the frustration and just ignore ...,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1.086896e+20,Melissa Demidov,1561477792923,1.0,"8:05 am I get there, sign says CLOSED even th...",None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1.129924e+20,Lisa Koehler,1571294526703,1.0,"I just wanted some iced tea and the lady said,...",None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1.007051e+20,Rudy Diaz,1581471097395,1.0,Stopped on the way out. The coffee was weak. ...,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1.066181e+20,Thomas Charley,1553614482083,1.0,Person behind counter was rude ....,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9,Castle Coffee,"Castle Coffee, Excalibur Hotel and Casino, 385...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833206,1.111813e+20,M. Del,1511403706460,2.0,Nice room but terrible night sleep. Continuou...,None,0x80c8c43b8995c0b5:0xe2ac56cdb7db8765,SB Las Vegas at Caesars Palace,"SB Las Vegas at Caesars Palace, Paradise, NV 8...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8833207,1.048418e+20,喵咪,1514964796392,1.0,Poor guest service. Waited for our luggages fo...,None,0x80c8c43b8995c0b5:0xe2ac56cdb7db8765,SB Las Vegas at Caesars Palace,"SB Las Vegas at Caesars Palace, Paradise, NV 8...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8833285,1.026047e+20,Kevin Diaz,1592202676820,1.0,Bad bottle of wine for $25,None,0x80c8c47be23c6541:0xe6c010eb30ac72b1,Mom and Pop's Liquor Store,"Mom and Pop's Liquor Store, 557 E Sahara Ave #...",None,...,Beer store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8833291,1.139538e+20,Teresa John,1566173405294,1.0,This place such llppppppppppppppppppppppppppkp...,None,0x80c8c47be23c6541:0xe6c010eb30ac72b1,Mom and Pop's Liquor Store,"Mom and Pop's Liquor Store, 557 E Sahara Ave #...",None,...,Beer store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
healthcare_categories = {
    "medical clinic", "women's health clinic", "home health care service", "internist", "allergist",
    "diabetes center", "doctor", "family practice physician", "optometrist", "pharmacy", "dentist",
    "medical supply store", "orthopedic surgeon", "surgeon", "psychotherapist", "dental clinic",
    "orthodontist", "physical therapist", "chiropractor", "hospital", "sleep clinic", "optician",
    "cancer treatment center", "medical center", "physical therapy clinic", "medical laboratory",
    "public health department", "psychologist", "radiologist", "podiatrist", "nursing home",
    "medical group", "x-ray lab", "dermatologist", "pediatrician", "massage therapist",
    "periodontist", "obstetrician-gynecologist", "sports medicine clinic", "psychiatric hospital",
    "orthotics & prosthetics service", "physical fitness program", "ophthalmologist", "hospice",
    "retirement home", "rehabilitation center", "pediatric dentist", "teeth whitening service",
    "dental hygienist", "endodontist", "dental implants periodontist", "cosmetic dentist",
    "emergency dental service", "urgent care center", "wellness center", "walk-in clinic",
    "general practitioner", "children's hospital", "mental health clinic", "otolaryngologist","drug testing service","fertility physician"
    ,"endocrinologist","health consultant", "surgical center","homeopathic pharmacy", "psychiatrist", "medical equipment supplier",
    "massage therapist", "eye care center","optician","medical laboratory","rehabilitation center"
}
# Function to check if a business belongs to the healthcare category
def classify_healthcare(row):
    for i in range(1, 16):  # Assuming category_1 to category_15 exist
        category_col = f"category_{i}"
        if category_col in row and isinstance(row[category_col], str):  # Check if column exists and is a string
            for keyword in healthcare_categories:
                if re.search(rf"\b{keyword}\b", row[category_col], re.IGNORECASE):  # Match whole word
                    return "Healthcare"  # Assign healthcare label
    return "Other"  # Default category if no match

# Apply the function to create a new 'general_labelling' column
negative_df["general_labelling"] = negative_df.apply(classify_healthcare, axis=1)

In [15]:
# Filter records where 'general_labelling' is "Healthcare"
healthcare_dataset = negative_df[negative_df["general_labelling"] == "Healthcare"]

# Define the file path to save the Healthcare dataset
nevada_healthcare_dataset_path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/nevada_healthcare_dataset.csv"

# Save the filtered dataset to CSV
healthcare_dataset.to_csv(nevada_healthcare_dataset_path, index=False)

In [17]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/nevada_healthcare_dataset.csv"
nv = pd.read_csv(path)
nv

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,general_labelling
0,1.017970e+20,thealien lives,1625610027050,1.0,"Horrible, couldn't even get an appointment eve...",NaN,0x80c8bfc4e544cb1b:0x8e5e96a5bb1bedf9,Retina Consultants of Nevada,"Retina Consultants of Nevada, 653 N Town Cente...",NaN,...,Eye care center,Optometrist,Ophthalmology clinic,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
1,1.017656e+20,Peggy McClain,1616005253285,2.0,Over 2 hours waiting and have yet to see the d...,NaN,0x80c8bfc4e544cb1b:0x8e5e96a5bb1bedf9,Retina Consultants of Nevada,"Retina Consultants of Nevada, 653 N Town Cente...",NaN,...,Eye care center,Optometrist,Ophthalmology clinic,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
2,1.171697e+20,Benjamin Cordova,1535126027325,1.0,If I could give CERO......The wait was too lon...,NaN,0x80c8bfc4e544cb1b:0x8e5e96a5bb1bedf9,Retina Consultants of Nevada,"Retina Consultants of Nevada, 653 N Town Cente...",NaN,...,Eye care center,Optometrist,Ophthalmology clinic,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
3,1.175180e+20,James Blades,1488233569800,2.0,You wait in a reception area full of waiting p...,NaN,0x80c8bfc4e544cb1b:0x8e5e96a5bb1bedf9,Retina Consultants of Nevada,"Retina Consultants of Nevada, 653 N Town Cente...",NaN,...,Eye care center,Optometrist,Ophthalmology clinic,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
4,1.058999e+20,Patricia James,1571959904384,2.0,The staff that answers the phone seems to be i...,NaN,0x80c8bfc4e544cb1b:0x8e5e96a5bb1bedf9,Retina Consultants of Nevada,"Retina Consultants of Nevada, 653 N Town Cente...",NaN,...,Eye care center,Optometrist,Ophthalmology clinic,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32721,1.111156e+20,Private Mail,1596641429281,1.0,This place never answers their phone.,"{'time': 1596744410634, 'text': ""We apologize ...",0x80c8c6b413e77675:0x79f42e8fc4dd38f9,Spay & Neuter Center of Southern Nevada - West,Spay & Neuter Center of Southern Nevada - West...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
32722,1.064233e+20,Greg Austin,1516483734798,1.0,Front desk was not very helpful when I called ...,"{'time': 1516484946213, 'text': 'Thank you for...",0x80c8c6b413e77675:0x79f42e8fc4dd38f9,Spay & Neuter Center of Southern Nevada - West,Spay & Neuter Center of Southern Nevada - West...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
32723,1.111016e+20,Andrew Wakefield,1598979090179,2.0,Take to long,NaN,0x80c8c6b413e77675:0x79f42e8fc4dd38f9,Spay & Neuter Center of Southern Nevada - West,Spay & Neuter Center of Southern Nevada - West...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
32724,1.038979e+20,Michael B,1555472934236,2.0,This place is the cheapest but bad service,NaN,0x80c8c6b413e77675:0x79f42e8fc4dd38f9,Spay & Neuter Center of Southern Nevada - West,Spay & Neuter Center of Southern Nevada - West...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
